In [ ]:
from pathlib import Path
import os

DAY = "20251201"
Version = "v1"

PROJECT_ROOT = Path(os.getenv("LLMSC_ROOT", ".")).resolve()

DATA_DIR = Path(os.getenv("LLMSC_DATA_DIR", PROJECT_ROOT / "input")).resolve()
OUT_DIR  = Path(os.getenv("LLMSC_OUT_DIR",  PROJECT_ROOT / "runs" / f"{DAY}.{Version}")).resolve()
OUT_DIR.mkdir(parents=True, exist_ok=True)

In [ ]:
RANDOM_SEED = 42
import random, os
import numpy as np
random.seed(RANDOM_SEED)
os.environ['PYTHONHASHSEED'] = str(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

print(f"🔒 Random seed set to {RANDOM_SEED} for reproducibility.")

🔒 Random seed set to 42 for reproducibility.


In [ ]:
import os
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY", "")
USE_LLM = bool(GEMINI_API_KEY)

if not USE_LLM:
    print("⚠️ GEMINI_API_KEY not set → LLM inference cells will be skipped.")

In [ ]:
import google.generativeai as genai
import scanpy as sc
import pandas as pd
import numpy as np
import scipy
import scipy.sparse
from scipy import io
import adjustText
import gc
import time
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.backends.backend_pdf import PdfPages
import seaborn as sns
from sklearn.metrics import confusion_matrix
from google.colab import userdata

import logging
logging.basicConfig(
    level=logging.INFO,
    format="%(levelname)s:%(name)s:%(message)s"
)

from llm_sc_curator import LLMscCurator
from llm_sc_curator.masking import FeatureDistiller

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# ==========================================
# 1. API
# ==========================================

genai.configure(api_key=GEMINI_API_KEY)
print("--- Available Models for your Key ---")
try:
    for m in genai.list_models():
        if 'generateContent' in m.supported_generation_methods:
            print(m.name)
except Exception as e:
    print(f"Error: {e}")

--- Available Models for your Key ---
models/gemini-2.5-flash
models/gemini-2.5-pro
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-exp-1206
models/gemini-2.5-flash-preview-tts
models/gemini-2.5-pro-preview-tts
models/gemma-3-1b-it
models/gemma-3-4b-it
models/gemma-3-12b-it
models/gemma-3-27b-it
models/gemma-3n-e4b-it
models/gemma-3n-e2b-it
models/gemini-flash-latest
models/gemini-flash-lite-latest
models/gemini-pro-latest
models/gemini-2.5-flash-lite
models/gemini-2.5-flash-image-preview
models/gemini-2.5-flash-image
models/gemini-2.5-flash-preview-09-2025
models/gemini-2.5-flash-lite-preview-09-2025
models/gemini-3-pro-preview
models/gemini-3-pro-image-preview
models/nano-banana-pro-preview
models/gemini-robotics-er-1.5-preview
models/gemini-2.5-computer-

In [ ]:
MODEL_NAME = 'models/gemini-2.5-pro'

# MODEL_NAME = 'models/gemini-3-pro-preview'
# MODEL_NAME = 'models/gemini-2.5-flash'
# MODEL_NAME = 'models/gemini-2.0-flash'

print(f"Using Model: {MODEL_NAME}")
model = genai.GenerativeModel(MODEL_NAME)

Using Model: models/gemini-2.5-pro


In [ ]:
save_path =  OUT_DIR / "cd4_benchmark_data.h5ad"

print(f"\n Loading data...: {save_path}")
adata= sc.read_h5ad(save_path)
print("\nCell counts per cluster (Should be balanced approx ~300 if 3 datasets merged):")
print(adata.obs['meta.cluster'].value_counts().head(10))


 Loading data...: /runs/20251201.v1/cd4_benchmark_data.h5ad

Cell counts per cluster (Should be balanced approx ~300 if 3 datasets merged):
meta.cluster
CD4.c01.Tn.TCF7          300
CD4.c06.Tm.ANXA1         300
CD4.c07.Tm.ANXA2         300
CD4.c16.Tfh.CXCR5        300
CD4.c14.Th17.SLC4A10     300
CD4.c15.Th17.IL23R       300
CD4.c12.Tem.GZMK         300
CD4.c10.Tm.CAPG          300
CD4.c20.Treg.TNFRSF9     300
CD4.c17.TfhTh1.CXCL13    300
Name: count, dtype: int64


In [ ]:
# ==========================================
# Correct Label Generation Function
# Definition based on cluster IDs from Zheng et al. 2021
# ==========================================

def get_cd4_ground_truth(cluster_name: str) -> str:
    """
    Map Zheng et al. CD4 meta.cluster names to GT categories.

    We intentionally keep this mapping simple and deterministic, based only on
    the author-provided cluster names (Tn/Tm/Tem/Temra/Th17/Tfh/Treg/ISG).
    Finer biological nuances are handled at the scoring layer, not here.
    """
    s = str(cluster_name).lower()

    # 1) Regulatory / helper subsets
    if any(k in s for k in ["treg", "foxp3", "regulatory"]):
        return "CD4_Treg"
    if any(k in s for k in ["tfh", "follicular", "cxcl13", "cxcr5"]):
        return "CD4_Tfh"
    if any(k in s for k in ["th17", "il17", "rorc", "il23r"]):
        return "CD4_Th17"

    # 2) ISG & cycling axes
    if any(k in s for k in ["isg", "interferon", "ifit1"]):
        return "CD4_ISG"
    if any(k in s for k in ["cycling", "proliferating", "mki67", "top2a"]):
        return "CD4_Cycling"

    if "mix" in s:
        return "CD4_Other"

    # 3) Exhausted pool (if ever present in CD4 cluster IDs)
    if any(k in s for k in ["tex", "exhausted", "pdcd1"]):
        return "CD4_Exhausted"

    # 4) Naive / memory pools
    if "tn." in s or "naive" in s:
        return "CD4_Tn.Naive"
    if "temra" in s:
        return "CD4_Temra.EffMem"
    if "tem." in s:
        return "CD4_Tem.EffMem"
    if "tm." in s:
        return "CD4_Tm.EffMem"

    # 5) Fallback
    return "CD4_Other"


print("Applying CD4 Ground Truth Mapping...")
adata.obs['GT_Category'] = adata.obs['meta.cluster'].apply(get_cd4_ground_truth)

print(adata.obs['GT_Category'].value_counts())


Applying CD4 Ground Truth Mapping...
GT_Category
CD4_Tm.EffMem       1967
CD4_Treg            1041
CD4_Tn.Naive         637
CD4_Tfh              600
CD4_Th17             600
CD4_Other            437
CD4_Tem.EffMem       300
CD4_Temra.EffMem     270
CD4_ISG              153
Name: count, dtype: int64


In [ ]:
GT_KEYWORDS_CD4 = {
    "CD4_Tem.EffMem":      ["GZMK", "CCL5", "ANXA1"],
    "CD4_Temra.EffMem":    ["CX3CR1", "KLRG1", "GZMB"],
    "CD4_Tn.Naive":        ["TCF7", "LEF1", "CCR7"],
    "CD4_Th17":            ["RORC", "CCR6", "IL23R"],
    "CD4_Tm.EffMem":       ["IL7R", "CCR7", "SELL"],
    "CD4_Treg":            ["FOXP3", "IL2RA", "IKZF2"],
    "CD4_ISG":             ["ISG15", "IFIT1", "MX1"],
    "CD4_Tfh":             ["CXCL13", "BCL6", "ICA1"],
    #"CD4_Th1":             ["TBX21", "IFNG", "CXCR3"],
    #"CD4_Exhausted":       ["HAVCR2", "LAG3", "TOX"],
    #"CD4_Cycling":         ["MKI67", "TOP2A"],

    # Catch-all
    #"CD4_Other":           [],
}

var_names = adata.var_names
filtered_markers = {k: [g for g in v if g in var_names] for k, v in GT_KEYWORDS_CD4.items()}

In [ ]:
if "highly_variable" not in adata.var.columns:
    print("[Setup] Computing global HVGs for CD8 benchmark...")

    hvg_kwargs = dict(
        n_top_genes=2000,
        subset=False,
        batch_key="Cancer_Type",
    )

    if "counts" in adata.layers:
        hvg_kwargs.update(
            flavor="seurat_v3",
            layer="counts",
        )
        print("[HVG] Using flavor='seurat_v3' on layers['counts'] with batch_key='Cancer_Type'.")
    else:
        hvg_kwargs.update(
            flavor="seurat",
        )
        print("[HVG] `layers['counts']` not found → using flavor='seurat' on log1p .X.")

    print(hvg_kwargs)

    sc.pp.highly_variable_genes(adata, **hvg_kwargs)

# Curator Initialize
curator = LLMscCurator(api_key=GEMINI_API_KEY, model_name=MODEL_NAME)
curator.set_global_context(adata)

[Setup] Computing global HVGs for CD8 benchmark...
[HVG] Using flavor='seurat_v3' on layers['counts'] with batch_key='Cancer_Type'.
{'n_top_genes': 2000, 'subset': False, 'batch_key': 'Cancer_Type', 'flavor': 'seurat_v3', 'layer': 'counts'}


In [ ]:
N_GENES = 50
benchmark_results = []

# 1) Retrieve cluster list from meta.cluster (sorted for reproducibility)
unique_clusters = sorted(adata.obs["meta.cluster"].unique())

# Clusters with ambiguous GT values are excluded from the main benchmark.
EXCLUDE_GT_LABELS = {"CD4_Other", "Other", "Unknown"}

cluster_meta = []
dropped_clusters = []

for c in unique_clusters:
    gt_label = get_cd4_ground_truth(c)
    if gt_label in EXCLUDE_GT_LABELS:
        dropped_clusters.append((c, gt_label))
        continue
    cluster_meta.append((c, gt_label))

print(f"🚀 Starting CD4 benchmark for {len(cluster_meta)} clusters (Top {N_GENES} genes)...")
if dropped_clusters:
    print("⚠️ Excluded ambiguous clusters from main benchmark:")
    for c, lab in dropped_clusters:
        print(f"   - {c} (GT={lab})")


def ensure_json_result(x):
    """Always normalize LLM responses to dict format. Also complete missing keys."""
    if isinstance(x, dict):
        return {
            "cell_type":  x.get("cell_type", "Unknown"),
            "confidence": x.get("confidence", "Low"),
            "reasoning":  x.get("reasoning", ""),
        }
    elif isinstance(x, str):
        return {
            "cell_type":  x,
            "confidence": "Low",
            "reasoning":  "",
        }
    else:
        return {
            "cell_type":  "Error",
            "confidence": "Low",
            "reasoning":  repr(x),
        }


# 2) Main loop: 1 row = 1 meta.cluster
for i, (cluster_name, gt_label) in enumerate(cluster_meta):
    gt_keywords = GT_KEYWORDS_CD4.get(gt_label, [])

    print(f"\n[{i+1}/{len(cluster_meta)}] Processing: {cluster_name} → {gt_label}")

    # -------------------------------------------------
    # A. Standard pipeline (no masking)
    # -------------------------------------------------
    adata.obs["binary_group"] = "Rest"
    adata.obs.loc[adata.obs["meta.cluster"] == cluster_name, "binary_group"] = "Target"

    try:
        sc.tl.rank_genes_groups(
            adata,
            groupby="binary_group",
            groups=["Target"],
            reference="Rest",
            method="wilcoxon",
            use_raw=False,
        )
        df_std = sc.get.rank_genes_groups_df(adata, group="Target")
        genes_std = df_std["names"].head(N_GENES).tolist()
    except Exception as e:
        print(f"[WARN] Standard DE failed for {cluster_name}: {e}")
        genes_std = []
    try:
        if genes_std:
            raw_std = curator.annotate(genes_std, use_auto_context=False)
        else:
            raw_std = {
                "cell_type": "NoGenes",
                "confidence": "Low",
                "reasoning": "Empty DEG list",
            }
        res_std = ensure_json_result(raw_std)
    except Exception as e:
        print(f"[WARN] Standard annotate failed for {cluster_name}: {e}")
        res_std = ensure_json_result(
            {"cell_type": "Error", "confidence": "Low", "reasoning": str(e)}
        )

    time.sleep(2)

    # -------------------------------------------------
    # B. Curated pipeline (LLM-scCurator, masking ON)
    # -------------------------------------------------
    try:
        genes_cur = curator.curate_features(
            adata,
            group_col="meta.cluster",
            target_group=cluster_name,
            n_top=N_GENES,
            use_statistics=True,   # Gini + Regex Masking ON
        )
    except Exception as e:
        print(f"[WARN] curate_features failed for {cluster_name}: {e}")
        genes_cur = []

    try:
        if genes_cur:
            raw_cur = curator.annotate(genes_cur)
        else:
            raw_cur = {
                "cell_type": "NoGenes",
                "confidence": "Low",
                "reasoning": "Curated gene list empty",
            }
        res_cur = ensure_json_result(raw_cur)
    except Exception as e:
        print(f"[WARN] Curated annotate failed for {cluster_name}: {e}")
        res_cur = ensure_json_result(
            {"cell_type": "Error", "confidence": "Low", "reasoning": str(e)}
        )

    time.sleep(2)

    # -------------------------------------------------
    # C. Save Results
    # -------------------------------------------------
    print(f"  👉 Std Ans: {res_std['cell_type'][:40]} ({res_std['confidence']})")
    print(f"  👉 Cur Ans: {res_cur['cell_type'][:40]} ({res_cur['confidence']})")

    benchmark_results.append(
        {
            "Cluster_ID": cluster_name,
            # Ground truth
            "Ground_Truth": gt_label,
            "Ground_Truth_Label": gt_label,
            "Ground_Truth_Keywords": ", ".join(gt_keywords),
            # Standard
            "Standard_Genes": ";".join(genes_std),
            "Standard_Answer": res_std["cell_type"],
            "Standard_CellType": res_std["cell_type"],
            "Standard_Confidence": res_std["confidence"],
            "Standard_Reasoning": res_std["reasoning"],
            # Curated
            "Curated_Genes": ";".join(genes_cur),
            "Curated_Answer": res_cur["cell_type"],
            "Curated_CellType": res_cur["cell_type"],
            "Curated_Confidence": res_cur["confidence"],
            "Curated_Reasoning": res_cur["reasoning"],
        }
    )

    if (i + 1) % 5 == 0:
        tmp_path = f"{outdir}/cd4_benchmark_progress.csv"
        pd.DataFrame(benchmark_results).to_csv(tmp_path, index=False)
        print(f"  💾 Progress saved to {tmp_path}")
    time.sleep(1)

df_results = pd.DataFrame(benchmark_results)
save_path = f"{outdir}/cd4_benchmark_results.csv"
df_results.to_csv(save_path, index=False)
print(f"\n✅ CD4 Benchmark Complete! Saved to {save_path}")

🚀 Starting CD4 benchmark for 22 clusters (Top 50 genes)...
⚠️ Excluded ambiguous clusters from main benchmark:
   - CD4.c23.Mix.NME1 (GT=CD4_Other)
   - CD4.c24.Mix.NME2 (GT=CD4_Other)

[1/22] Processing: CD4.c01.Tn.TCF7 → CD4_Tn.Naive
  👉 Std Ans: Low-quality cell (High)
  👉 Cur Ans: Naive T cell (High)

[2/22] Processing: CD4.c02.Tn.PASK → CD4_Tn.Naive
  👉 Std Ans: Early Activated T cell (High)
  👉 Cur Ans: T follicular helper cell (High)

[3/22] Processing: CD4.c03.Tn.ADSL → CD4_Tn.Naive
  👉 Std Ans: Naive T cell (High)
  👉 Cur Ans: Naive T cell (High)

[4/22] Processing: CD4.c04.Tn.il7r → CD4_Tn.Naive
  👉 Std Ans: Unidentified cell (Proliferating) (Low)
  👉 Cur Ans: T cell (proliferating) (Medium)

[5/22] Processing: CD4.c05.Tm.TNF → CD4_Tm.EffMem
  👉 Std Ans: Effector T cell (High-Mito) (High)
  👉 Cur Ans: CD8+ Temra T cell (High)
  💾 Progress saved to /runs/20251201.v1/cd4_benchmark_progress.csv

[6/22] Processing: CD4.c06.Tm.ANXA1 → CD4_Tm.EffMem
  👉 Std Ans: Naive T cell (High)